<a href="https://colab.research.google.com/github/imkeertiag/RagPipeline/blob/main/Rag_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/imkeertiag/RagPipeline.git

Cloning into 'RagPipeline'...


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Percipio -Kubernetes.pdf to Percipio -Kubernetes.pdf


In [ ]:
!pip install -U langchain langchain-community pypdf
!pip install -U langchain langchain-community
!pip install -U langchain langchain-community langchain-text-splitters pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Percipio -Kubernetes.pdf")
docs=loader.load()


In [ ]:
# print(docs)
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)
documents = text_split.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'macOS Version 26.0.1 (Build 25A362) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20251122044150Z00'00'", 'title': 'Percipio -Kubernetes', 'moddate': "D:20251122044150Z00'00'", 'source': 'Percipio -Kubernetes.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='Percipient - course Pods ….summary\n1. Identify what a Pod is & Explain its crucial role in Kubernetes\nWhat is a Pod?\nA Pod is the smallest deployable and schedulable unit in Kubernetes.\u2028\nIt acts as a wrapper around:\n• One container (most common case), or\n• Multiple tightly coupled containers that must run together\nKey characteristics\n• Shared execution environment → containers inside a pod share:\n◦ Storage volumes\n◦ Network namespace (same IP & port space)\n◦ IPC (inter-process communication)\n• Pods represent a logical host for an application.\n• Containers inside a pod are always:\n◦ Co-located (run on the same node)\n◦ Co-scheduled\n◦ Share resource

In [ ]:
# The Ollama API is now available at 127.0.0.1:11434.
# sudo apt-get install zstd
# curl -fsSL https://ollama.com/install.sh | sh
# ollama --version
# ollama serve &
# !pip install ollama
# !ollama pull qwen2.5:3b

!ollama pull nomic-embed-text

from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"
)

# create embeddings
# vector = embedding.embed_query("Kubernetes pod lifecycle")
# len(vector)

# chromadb ,, faiss-cpu
from langchain_community.vectorstores import Chroma
data = Chroma.from_documents(documents , embedding)

# print(data)

In [ ]:
#now last step adding an LLM
from langchain_community.llms import Ollama
# print("LangChain + Ollama import successful")


#!ollama pull deepseek-r1:7b
# !ollama pull qwen2.5:7b
!ollama pull llama3:8b
!ollama list


NAME                       ID              SIZE      MODIFIED               
llama3:8b                  365c0bd3c000    4.7 GB    Less than a second ago    
qwen2.5:7b                 845dbda0ea48    4.7 GB    11 minutes ago            
deepseek-r1:7b             755ced02ce7b    4.7 GB    18 minutes ago            
nomic-embed-text:latest    0a109f422b47    274 MB    49 minutes ago            
qwen2.5:3b                 357c53fb659c    1.9 GB    About an hour ago         


In [ ]:
llm = Ollama(
    model="deepseek-r1:7b",
    temperature=0.5,
)
questions = [
    "What is jenkins?",
    "What is Kubernetes pods?"
    ]

for query in questions:
    print("Q:", query)
    docs1 = data.similarity_search(query)
    context1 = "\n\n".join([doc.page_content for doc in docs1])

    prompt1 = f"""
    Answer the question using the context below.
    Context:
    {context1}

    Question:
    {query}
    """

    answer1 = llm.invoke(prompt1)
    print("A: ", answer1)
    print("------------------------------------------deepseek-r1:7b----------temp = 0.1-------------------------------------------------------------")


Q: What is jenkins?
A:  The question asks about Jenkins in the context provided, which focuses on Kubernetes architecture, services, and Ingress controllers. However, Jenkins is a separate entity—it's an open-source tool used for CI/CD (Continuous Integration/Continuous Deployment) and automating software testing. It isn't related to Kubernetes or networking within clusters as discussed in the context.

**Answer:**  
Jenkins is not mentioned in the provided context and is unrelated to Kubernetes or cloud-native architecture. Jenkins is a separate tool primarily used for continuous integration and deployment processes, serving a different purpose outside of Kubernetes environments.
------------------------------------------deepseek-r1:7b----------temp = 0.1-------------------------------------------------------------
Q: What is Kubernetes pods?
A:  Kubernetes pods are individual containers within a Kubernetes cluster, each specified by a PodSpec containing details such as selector, repl

In [ ]:
llm = Ollama(
    model="qwen2.5:3b",
    temperature=0.1,
)
questions = [
    "What is jenkins?",
    "What is Kubernetes pods?"
    ]

for query in questions:
    print("Q:", query)
    docs1 = data.similarity_search(query)
    context1 = "\n\n".join([doc.page_content for doc in docs1])

    prompt1 = f"""
    Answer the question using the context below.
    Context:
    {context1}

    Question:
    {query}
    """

    answer1 = llm.invoke(prompt1)
    print("A: ", answer1)
    print("----------------------------------qwen2.5:3b------------------temp = 0.1-------------------------------------------------------------")



Q: What is jenkins?
A:  The provided context does not contain any information about Jenkins. The context appears to be related to Kubernetes, including services, ingress controllers, and how they work together in a cluster. Therefore, I cannot answer the question "What is jenkins?" based on the given text. Jenkins is an entirely different tool used for building, testing, and deploying software.
-----------------------------------------------------------------------------------------------------------------
Q: What is Kubernetes pods?
A:  Kubernetes pods are the fundamental units of deployment in Kubernetes. They represent a collection of containers that work together to serve a specific purpose or provide a certain service. Each pod contains an application, along with any necessary resources like volumes and environment variables.

Here's how pods are created and managed within Kubernetes:

1. **Pod Spec**: A pod is defined by its spec, which includes the container images to be run, re

In [ ]:
llm = Ollama(
    model="llama3:8b",
    temperature=0.1,
)
questions = [
    "What is jenkins?",
    "What is Kubernetes pods?",
    "Name all the topics of the document?"
    ]

for query in questions:
    print("Q:", query)
    docs1 = data.similarity_search(query)
    context1 = "\n\n".join([doc.page_content for doc in docs1])

    prompt1 = f"""
    Answer the question using the context below.
    Context:
    {context1}

    Question:
    {query}
    """

    answer1 = llm.invoke(prompt1)
    print("A: ", answer1)
    print("--------------------------------llama3:8b--------------------temp = 0.1 for above-------------------------------------------------------------")


llm = Ollama(
    model="llama3:8b",
    temperature=0.5,
)
questions = [
    "What is jenkins?",
    "What is Kubernetes pods?",
    "Name all the topics of the document?"
    ]

for query in questions:
    print("Q:", query)
    docs1 = data.similarity_search(query)
    context1 = "\n\n".join([doc.page_content for doc in docs1])

    prompt1 = f"""
    Answer the question using the context below.
    Context:
    {context1}

    Question:
    {query}
    """

    answer1 = llm.invoke(prompt1)
    print("A: ", answer1)
    print("----------------------------llama3:8b-------------------------temp = 0.5 for above------------------------------------------------------------")



Q: What is jenkins?
A:  There is no mention of Jenkins in the provided context. The text only discusses Kubernetes, services, and ingress controllers. Therefore, I cannot answer the question about what Jenkins is based on this context. If you meant to provide a different context or clarify your question, please feel free to do so!
--------------------------------llama3:8b--------------------temp = 0.1 for above-------------------------------------------------------------
Q: What is Kubernetes pods?
A:  Based on the provided context, there is no specific mention of "Kubernetes pods" in the text. However, we can infer that a pod is a fundamental unit of deployment in Kubernetes, as it is mentioned alongside other concepts like ReplicaSets and Services.

A pod is likely a single instance of a running application or service, which can be replicated using a ReplicaSet. Pods are identified by labels (selector) and have an ownerReference linking them back to their ReplicaSet.
----------------